# Figure 2

In [1]:
from functools import partial

from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio

from agent.Vampire import SimpleVampire, SmartVampire, SmartDynamicVampire
from model.VampireModel import VampireModel 

In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be re

In [2]:
BaseModel = partial(
    VampireModel,
    n_roots = 10,
    root_size = 15,
    max_iteration = 5000, 
    hunt_probability = 0.91,
    food_sharing = True,
    reproduction = True,
    reproduction_probability = 0.5
)

In [13]:
def plot(xlabel, ylabel, legend1, legend2,legend3, df1, df2, df3, file):
    layout = go.Layout(
        xaxis=dict(
            title=xlabel,
            linecolor="#BCCCDC", 
            showgrid=False  
        ),
        yaxis=dict(
            title=ylabel,  
            linecolor="#BCCCDC",  
            showgrid=False,  
        )
    )
    fig = go.Figure([
        go.Scatter(
            name=legend2,
            x=df2.index,
            y=df2['mean'],
            mode='lines',
            line=dict(color='#c6dc5a', width=3), 
        ),
        go.Scatter(
            name=legend1,
            x=df1.index,
            y=df1['mean'],
            mode='lines',
            line=dict(color='#370725', width=3), 
        ),
         go.Scatter(
            name=legend3,
            x=df3.index,
            y=df3['mean'],
            mode='lines',
            line=dict(color='#f07ace', width=3), 
        )
    ], layout=layout)


    fig.update_layout(
        template="plotly_dark", 
        plot_bgcolor='#a38285', 
        paper_bgcolor='#a38285', 
        font_size=16,
        
    )
    fig.update_yaxes(range=[0, max(df1['mean'].max(), df2['mean'].max(), df3['mean'].max())])
    fig.write_image(file)
    fig.show()


In [4]:
MC = 100

In [5]:
df_simple = pd.DataFrame()
for i in tqdm(range(MC)):
    model = BaseModel(
        vampire_type = SimpleVampire
    )    
    model.run_model()
    df_simple.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_simple['mean'] = df_simple.mean(axis=1)
df_simple['std'] = df_simple.std(axis=1)

100%|██████████| 100/100 [01:01<00:00,  1.63it/s]


In [6]:
df_smart = pd.DataFrame()
for i in tqdm(range(MC)):
    model = BaseModel(
        vampire_type = SmartVampire
    )    
    model.run_model()
    df_smart.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_smart['mean'] = df_smart.mean(axis=1)
df_smart['std'] = df_smart.std(axis=1)

100%|██████████| 100/100 [00:29<00:00,  3.34it/s]


In [7]:
df_dynamic = pd.DataFrame()
for i in tqdm(range(MC)):
    model = BaseModel(
        vampire_type = SmartDynamicVampire,
        max_motivation = 7
    )    
    model.run_model()
    df_dynamic.insert(loc=i, column=str(i), value=model.datacollector.get_model_vars_dataframe().squeeze())

df_dynamic['mean'] = df_dynamic.mean(axis=1)
df_dynamic['std'] = df_dynamic.std(axis=1)

100%|██████████| 100/100 [07:16<00:00,  4.37s/it]


In [14]:
plot('Iteracja','Liczba nietoperzy', 'Simple', 'Smart', 'Dynamic', df_simple, df_smart, df_dynamic, 'results/simple_smart_dynamic.jpg')